# Review of Day 1

During Day 1, we introduced Julia basics, and saw that Julia has the following features:

- Familiar syntax
- Easy to make generic code ("write once, run everywhere")
- Julia is fast

Day 2 will be about Performance and Parallelism.
We will see how to profile code, some performance gotchas, and several ways to parallelize code using Julia.

# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [ ]:
numsteps = 1000
pos = 0 
for j in 1:numsteps
            
    if rand() < 0.5
        step = -1
    else
        step = +1
    end
            
    pos += step 
end

The code seems to execute almost instantaneously, but we should **profile** (time) it:

In [ ]:
@time begin
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
end

Although it's fast, it seems to be allocating memory unexpectedly.

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [ ]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

In [ ]:
@time walk(1000)

In [ ]:
@time walk(1000)

Messages: 
- Wrap everything in a function
- Run the function once, before timing only on the *second* run

Now we run it several times to collect data:

In [ ]:
numsteps   = 1000
numwalkers = 10000

@time data = [walk(numsteps) for i in 1:numwalkers]  # final positions

A population of simple random walks should have mean $0$ and variance equal to the total time. Let's check it:

In [ ]:
mean(data)

In [ ]:
var(data), numsteps

In [ ]:
≈(var(data), numsteps, rtol=1e-2)

We can plot the histogram:

In [ ]:
using Plots; gr()

histogram(data, nbins=100)

Again, however, the high number of allocations is suspect -- in Julia, this is usually a warning that there is a "type instability". We again try wrapping it in a function, even though it seems so simple:

In [ ]:
function run_walks(numwalkers, numsteps)

    data = [walk(numsteps) for i in 1:numwalkers]

    return data
end

In [ ]:
numsteps   = 1000
numwalkers = 10000

data = run_walks(1, 1)  # compile the function

# now profile it:
@time data = run_walks(numwalkers, numsteps);  

In [ ]:
var(data)

# DistributedArrays 

Jump to [notebook 1a.](1a. Basics of distributed arrays.ipynb) for the basics of distributed arrays in Julia.

# Basic parallelism

In [ ]:
addprocs(2)

In [ ]:
@everywhere using DistributedArrays

In [ ]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

In [ ]:
walkers = distribute(1:numwalkers);

In [ ]:
walkers.indexes

In [ ]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end

walkers = distribute(1:numwalkers);

@time positions = map( _ -> walk(numsteps), walkers)

In [ ]:
positions

In [ ]:
mean(positions)

In [ ]:
var(positions)

In [ ]:
squared_positions = map(x->x^2, positions);

In [ ]:
mean(squared_positions) ≈ var(positions)

# Another example: random matrices

In [ ]:
using Plots; gr()

In [ ]:
addprocs(4)

In [ ]:
@everywhere begin
    using DistributedArrays
    using StatsBase
    using Plots
end

In [ ]:
@everywhere function stochastic(β = 2, n = 200)
    h = n ^ -(1/3)
    x = 0:h:10
    N = length(x)
    d = (-2 / h^2 .- x) + 2*sqrt(h*β) * randn(N) # diagonal
    e = ones(N - 1) / h^2                     # subdiagonal
  
    eigvals(SymTridiagonal(d, e))[N]        # smallest negative eigenvalue
end

Serial version:

In [ ]:
println("Serial version")

t = 10000
p = plot()
for β = [1,2,4,10,20]
    
    z = fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end
p

A related parallel construct: `@parallel`. This does a "reduce" operation.

In [ ]:
println("@parallel version")

@everywhere t = 10000

p = plot()

for β = [1,2,4,10,20]
    
    z = @parallel (+) for p = 1:nprocs()
        fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    end
    
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end

p

In [ ]:
function dhist(x; closed=:left, nbins=10)
    
    hist_parts = DArray(p->fit(Histogram, localpart(x), closed=closed, nbins=nbins).weights, (nbins*length(x.pids),))
    
    reduce(+, map(pid -> @fetchfrom(pid, localpart(hist_parts)), hist_parts.pids))
      
end

In [ ]:
a = randn(10000)
d = distribute(a)

dhist(d)